In [1]:
import os
from dotenv import load_dotenv
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI

# Load API keys and set up tracing
load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGSMITH_PROJECT"] = "Intro to LangGraph"

# --- Define the State Schema with a Reducer ---
# By using Annotated and operator.add, we're telling the graph
# that any new 'messages' should be added to the existing list,
# not replace it. This is a "reducer".
class GraphState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

# --- Define Nodes ---
def call_llm(state: GraphState):
    # This node just calls the LLM with the current list of messages
    llm = ChatOpenAI(model="gpt-4o")
    response = llm.invoke(state['messages'])
    # The new message is returned, and the reducer will add it to the state
    return {"messages": [response]}

# --- Build the Graph ---
workflow = StateGraph(GraphState)

# Add the single node
workflow.add_node("llm", call_llm)

# Set the entry point and exit point
workflow.set_entry_point("llm")
workflow.add_edge("llm", END)

# Compile the graph
app = workflow.compile()

# --- Run the Graph ---
# We'll run it twice to show how the state is accumulated

print("--- First Run ---")
# The initial input is a list with one message
initial_state = {"messages": [HumanMessage(content="Hi! I'm Bob.")]}
final_state_1 = app.invoke(initial_state)
print(final_state_1['messages'])

print("\n--- Second Run ---")
# For the second run, we pass the final state from the first run.
# The reducer will add the new messages to this existing list.
final_state_2 = app.invoke(final_state_1)
print(final_state_2['messages'])

--- First Run ---
[HumanMessage(content="Hi! I'm Bob.", additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_cbf1785567', 'id': 'chatcmpl-CTpSUBeDJDfdIfAwEFcDdJ9dwsK38', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--a76e8b70-9e26-4c50-98c9-6438b118c849-0', usage_metadata={'input_tokens': 12, 'output_tokens': 10, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

--- Second Run ---
[HumanMessage(